In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.0 MB/s eta 0:00:00


In [ ]:
from Bio import Entrez
Entrez.email = "noel22338@iiitd.ac.in"
handle = Entrez.einfo()
record = Entrez.read(handle)

In [ ]:
def download_fasta(email, accession_number, output_file):
    Entrez.email = email
    handle = Entrez.efetch(db = 'nucleotide', id = accession_number, rettype = 'fasta', retmode = 'text')
    sequence = handle.read()
    with open(output_file, 'w') as f:
        f.write(sequence)

# The above function takes arguments and and writes the sequence to an output file

In [ ]:
email = "noel22338@iiitd.ac.in"
chromosome_12_id = 'NC_001144.5'
chromosome_7_id = 'NC_001139.9'
download_fasta(email, chromosome_12_id, "chromosome12.fasta")
download_fasta(email, chromosome_7_id, "chromosome7.fasta")

Based on this [database](https://https://yeastgenome.org/locus/S000004034/sequence) I found out the indices and the respective chromosomes for PDC1, PDC5, and PDC6. I had previously downloaded the chromosome12 and chromosome7 sequence, so using these fasta files I was able to extract the sequences for the PDC genes and stored them in a dictionary called "gene_sequences".

In [ ]:
from Bio import SeqIO
gene_sequences = {}
for seq_record in list(SeqIO.parse(f"chromosome12.fasta", "fasta")):
    gene_sequences['PDC1'] = seq_record.seq[232389:234081]
    gene_sequences['PDC5'] = seq_record.seq[410722:412414]

for seq_record in list(SeqIO.parse(f"chromosome7.fasta", "fasta")):
    gene_sequences['PDC6'] = seq_record.seq[651289:652981]

In [ ]:
from Bio import Seq
for pdc, seq in gene_sequences.items():
  print(f"{pdc}:   {len(seq)}     {seq}")

print("********************************************")
print("Reverse complement of PDC 1: " + Seq.reverse_complement(gene_sequences['PDC1']))
print("Reverse complement of PDC 6: " + Seq.reverse_complement(gene_sequences['PDC6']))


PDC1:   1692     TTATTGCTTAGCGTTGGTAGCAGCAGTCAACTTAGCTTGTTCAACCAAGTTTTGTGGAGCATCGAAGACTGGCAACATGATTTCAATCATTCTGATCTTAGAGTTGTCGTTGAAAGACTTGTCTTGGGTCAACTTGTCCCATTCACCGGTGGTAGCGACTCTGTGGGTTTCATAGTCCTTAGCACCGAAAGTTGGCAACAAGGATAGGTGGTCCCAACCTTGAATTTCGTTGTATTGAGCCTTTGGACCGTGAATCAACTTTTCAATGGTGTAACCATCGTTGTTCAAGACGAACAAGTATGGCTTCAAGCCCCATCTGATCATGGTGGAGATTTCTTGAACAGTCAATTGCAAAGAACCGTCACCAATGAATAAGATAACTCTCTTCTTTGGATCAATTTCTTCAGCAGCGAAAGCAGCACCCAAGGTAGCACCAGTGGTGAAACCAATGGAACCCCATAAGACTTGAGAGATACCGTAGGTGTTGTTTGGGAAAGTGGTTTGGTTGATACCGAAAGCGGAGGTACCGGTTTCAGCAATGACAACATCACCTTCTTGCAAGAAGTTACCCAATTGGTTCCACATCCATTCTTGCTTCAATGGGGTAGAAGCTGGGACAGCAGCGTTAGCTGGAGTTCTAGCTGGGACAGCAACTGGCTTGTAACCCTTAGCGGCGTCAGCAATAGTGGTCAACAACTTTTGCAAAACGAATTTCATTTGGACACCTGGGAAAGTGGCGTTTCTGATCTTCATGTGGTCGGAGTGGAATTCGACAATGTTCTTGGTCTTGTAAGAGTAAGAGAAAGAACCGGTGTTGAAATCAGACAACAAAGCACCGACAGACAAAATCAAGTCAGCAGATTCAACGGCTTCCTTAACTTCTGGCTTGGACAAGGTACCGACGTAAACACCACCGTATCTTGGGTGTTGTTCGTCAATGGAACCCTTACCCATTGGGGTGACGAAAGCTGGGAATTGAGTCA

The below ***translate*** function takes the File IO object, the subject sequence and the query sequence as parameters. It writes the DNA sequences, as well as the translated sequences into the file. It makes sure there are no gaps in the sequences.

In [ ]:
from Bio import Seq

def translate(myfile, main_sequence, pdc_sequence, title):

  if (main_sequence.count("-") == 0 and pdc_sequence.count("-") == 0):

    if (len(main_sequence) % 3 == 0):
      myfile.write(f"Title: {title}\n")
      myfile.write(f"Main seq: {main_sequence}\n")
      myfile.write(f"PDC seq: {pdc_sequence}\n")
      myfile.write(f"Main translated seq: {Seq.translate(main_sequence)}\n")
      myfile.write(f"PDC translated seq: {Seq.translate(pdc_sequence)}\n")
      myfile.write(f"{'*' * 1692}\n")


The below ***SNP_writer*** function takes File IO object, hsp (Highest Scoring Pair), and the count (how many hsps in that particular element). "hsp" is an object that contains attributes like:
1. **hsp.score** -> Score of the alignment
2. **hsp.expect** -> Expectation value
3. **hsp.query** -> The query sequence you performed blast with
4. **hsp.sbjct** -> A particular section of a strain against with alignment was performed.

In [ ]:
def SNP_writer(myfile, hsp, count):
  main_sequence = hsp.sbjct
  pdc_sequence = hsp.query


  myfile.write(f"HSP count: {count}\n");
  myfile.write(f"HSP score: {hsp.score}\n")
  myfile.write(f"HSP expect: {hsp.expect}\n")
  myfile.write(f"HSP start: {hsp.sbjct_start}\n")
  myfile.write(f"HSP end: {hsp.sbjct_end}\n")
  myfile.write(f"Length of seq: {len(pdc_sequence)}\n")

  count = 0

  for i in range(len(main_sequence)):
    if main_sequence[i] != pdc_sequence[i]:
      count+=1
      myfile.write(f"SNP {count} at pos {i+1}: {pdc_sequence[i]} -> {main_sequence[i]}\n")


  myfile.write("----------------------\n")


The below function ***blast_routine*** takes the PDC_gene_no as input and this can take values PDC1, PDC5 an PDC6. Based on this input, it writes to specific output files having a certain nomenclature and hsps are found out.

In [ ]:
def blast_routine(PDC_gene_no):

  E_VALUE_THRESH = 1e-4
  myfile = open(f"snp_results {PDC_gene_no}.txt", "w")
  file2 = open(f"translate_results {PDC_gene_no}.txt","w")
  blast_records = NCBIXML.parse(open(f"blast_results {PDC_gene_no}.xml"))
  alignment_count = 0
  for record in blast_records:
      for alignment in record.alignments:
        myfile.write(f"Title: {alignment.title}\n")
        alignment_count += 1
        count = 0
        for hsp in alignment.hsps:
          if (hsp.expect < E_VALUE_THRESH):
            count+=1;
            SNP_writer(myfile,hsp, count)
            translate(file2,hsp.sbjct,hsp.query, alignment.title)



Based on research I performed I found out that PDC1 and PDC6 have to be reverse complemented as these genes lie on the opposite strands to perform blast for the same. I realised this from this:

1. [PDC1](https://yeastgenome.org/locus/S000004034/sequence)
2. [PDC5](https://www.yeastgenome.org/locus/S000004124/sequence)
3. [PDC6](https://www.yeastgenome.org/locus/S000003319/sequence)

In [ ]:
E_VALUE_THRESH = 1e-4
from Bio.Blast import NCBIXML,NCBIWWW

for pdc_no, sequence_data in gene_sequences.items():

  if pdc_no in ['PDC1','PDC6']:
    reverse_complement_data = Seq.reverse_complement(sequence_data)
    result_handle = NCBIWWW.qblast("blastn", "nt", reverse_complement_data)

  else:
    result_handle = NCBIWWW.qblast("blastn", "nt", sequence_data)

  with open(f"blast_results {pdc_no}.xml","w") as xmlfile:
    blast_results = result_handle.read()
    xmlfile.write(blast_results)

In [ ]:
#After XML files are prepared, finding SNPs and translation are performed.
blast_routine('PDC1')
blast_routine('PDC5')
blast_routine('PDC6')

There is a report attached PolyPhen-2 report PDC1.pdf.

This is report for amino acid substitution for:

**Title**: gi|1586061137|gb|CP036478.1| Saccharomyces cerevisiae strain ySR128 chromosome XII, complete sequence

**Positions**
410631 - 412322

**PDC Amino Acid**

At position 15 I noticed there was a mutation from K -> S. I used [Polyphen-2](http://genetics.bwh.harvard.edu/ggi/pph2/757f2671575867bab9da8be0c3509da251312f4b/9647091.html)